<a href="https://colab.research.google.com/github/deuojen/fcc/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:

# split data into x (features) and y (labels)
x = dataset.copy()
y = x.pop('expenses')

# Split into train and test sets
train_dataset_raw, test_dataset_raw, train_labels, test_labels = train_test_split(x, y, test_size=0.2, random_state=42)

# check the lengths
len(train_dataset_raw), len(test_dataset_raw)

In [ ]:
# Create a column transformer
ct = make_column_transformer(
    # normalize all numerical columns to values between 0 and 1
    (MinMaxScaler(), ['age', 'bmi', 'children']),
    # one-hot encode the categorical columns
    (OneHotEncoder(handle_unknown='ignore'), ['sex', 'smoker', 'region'])
)

# fit the column transformer on the training data
ct.fit(train_dataset_raw)

# Transform the training and test data
train_dataset = ct.transform(train_dataset_raw)
test_dataset = ct.transform(test_dataset_raw)

In [ ]:
# Set up EarlyStopping to stop training when loss stops improving
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Build a neural network
model = keras.models.Sequential([
    layers.Dense(100),
    layers.Dense(10),
    layers.Dense(1)
])

# Compile the model
model.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['mae', 'mse'])

# Train the model
history = model.fit(train_dataset, train_labels, epochs=1000, callbacks=[callback])

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
